# Tucker Tensors

Tucker format is a decomposition of a tensor X as the product of a core tensor G and matrices (e.g., A,B,C) in each dimension. In other words, a tensor X is expressed as:

$
\mathcal{X} = \mathcal{G} \times_1 A \times_2 B \times_3 C
$

In MATLAB notation: `X=ttm(G,{A,B,C})`

In [ ]:
import pyttb as ttb
import numpy as np

In [ ]:
# Upcoming ttensors will be generated with this same initialization. 
def generate_sample_ttensor() -> ttb.ttensor:
    np.random.seed(0)
    core = ttb.tensor(np.random.rand(3,2,1), shape=(3,2,1)) # The core tensor.
    U = [np.random.rand(5,3), np.random.rand(4,2), np.random.rand(3,1)] # The factor matrices.
    X = ttb.ttensor(core, U) # Create the ttensor.
    
    return X

## Creating a ttensor with a tensor core

In [ ]:
X = generate_sample_ttensor() # Create the ttensor.
X

## Alternate core formats: sptensor or tensor

In [ ]:
np.random.seed(0)
sptensor_core = ttb.sptenrand([3,2,1], nonzeros=3) # Create a 3 x 2 x 1 sptensor.
U = [np.random.rand(5,3), np.random.rand(4,2), np.random.rand(3,1)] # The factor matrices.
Y = ttb.ttensor(sptensor_core, U) # Core is a sptensor.
Y

In [ ]:
dense_tensor = ttb.tensor(np.random.rand(3,2,1), (3,2,1)) # Core is a tensor.
U = [np.random.rand(5,3), np.random.rand(4,2), np.random.rand(3,1)] # The factor matrices.
Y = ttb.ttensor(dense_tensor, U) # Create the ttensor.
Y

## Creating a one-dimensional ttensor

In [ ]:
np.random.seed(0)
dense_tensor = ttb.tensor(2*np.random.rand(2,1), (2,)) # Core tensor.
Z = ttb.ttensor(dense_tensor, [np.random.rand(4,2)]) # One-dimensional ttensor.
Z

## Constituent parts of a ttensor

In [ ]:
X = generate_sample_ttensor() # Create the ttensor.
X.core # Core tensor.

In [ ]:
X.factor_matrices # List of matrices.

## Creating a ttensor from its constituent parts

In [ ]:
X = generate_sample_ttensor() # Create the ttensor.
Y = ttb.ttensor(X.core, X.factor_matrices) # Recreate a ttensor from its parts.
Y

## Creating an empty ttensor

In [ ]:
X = ttb.ttensor() # Empty ttensor.
X

## Use full or to_tensor to convert a ttensor to a tensor

In [ ]:
X = generate_sample_ttensor() # Create a ttensor.
X

In [ ]:
X.full() # Converts to a tensor.

In [ ]:
X.to_tensor() # Also converts to a tensor.

## Use reconstruct to compute part of a full tensor

In [ ]:
X = generate_sample_ttensor() # Create a ttensor.
X.reconstruct(1,2) # Extract first front slice.

## Use double to convert a ttensor to a (multidimensional) array

In [ ]:
X = generate_sample_ttensor() # Create the ttensor.
X.double() # Converts to an array.

## Use ndims and size to get the size of a ttensor

In [ ]:
X = generate_sample_ttensor() # Create the ttensor.
X.ndims # Number of dimensions.

In [ ]:
X.shape # Row vector of the sizes.

In [ ]:
X.shape[1] # Size of the 2nd mode.

## Subscripted reference for a ttensor

In [ ]:
X = generate_sample_ttensor() # Create the ttensor.
X.core[0,0,0] # Access an element of the core.

In [ ]:
X.factor_matrices[1] # Extract a matrix.

## Subscripted assignment for a ttensor

In [ ]:
X = generate_sample_ttensor() # Create a ttensor.
X.core = ttb.tenones(X.core.shape) # Insert a new core.
X

In [ ]:
X.core[1,1,0] = 7 # Change a single element.
X

In [ ]:
X.factor_matrices[2][0:2,0] = [1,1] # change slice of factor matrix
X.factor_matrices[2]

## Last index

In [ ]:
X.core[-1] # last element of core

In [ ]:
X.factor_matrices[-1]  # last factor matrix

In [ ]:
X.factor_matrices[-1][-1]  # last element of last factor matrix

## Basic operations (uplus, uminus, mtimes, etc.) on a ttensor

In [ ]:
X = generate_sample_ttensor() # Create ttensor.

### Addition

In [ ]:
+X # Calls uplus.

### Subtraction

In [ ]:
-X # Calls uminus.

### Multiplication

In [ ]:
5*X # Calls mtimes.

## Use permute to reorder the modes of a ttensor

In [ ]:
X = generate_sample_ttensor() # Create ttensor.
X.permute(np.array([2,1,0]))

## Displaying a ttensor

The ttensor displays by displaying the core and each of the component matrices.

In [ ]:
X = generate_sample_ttensor() # Create ttensor.
print(X)

In [ ]:
X # In the python interface